# Pre Processing Data for Machine Learning

## Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import boxcox
from scipy.stats import yeojohnson
from sklearn.preprocessing import QuantileTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

## Load the dataset

In [ ]:
bigmart_data = pd.read_csv('bigmartsales.csv')
bigmart_data.head()

## 1(a) Exploratory Data Analysis

First, we'll start with a general overview of the dataset.

In [ ]:

bigmart_data.describe(include='all')


### Here are the descriptive statistics for the BigMart dataset:

**Item_Identifier:** There are 1,559 unique item identifiers, with the most frequent one being "FDW13".

**Item_Weight:** The weights of items range from 0 to 21.35, with an average weight of approximately 11.68.

**Item_Fat_Content:** This categorical feature has 5 unique values, with "Low Fat" being the most frequent.

**Item_Visibility:** The average visibility is 0.066, and it ranges from 0 to 0.328.

**Item_Type:** There are 16 unique item types, with "Fruits and Vegetables" being the most frequent.

**Item_MRP:** The maximum retail price (MRP) of items ranges from 31.30 to 266.90, with an average of about 141.

**Outlet_Identifier:** There are 10 unique outlets, with "OUT027" being the most frequent.

**Outlet_Establishment_Year:** The outlets were established between 1985 and 2009.

**Outlet_Size:** This categorical feature has 3 unique sizes, with "Medium" being the most frequent. However, there are missing values in this column.

**Outlet_Location_Type:** There seems to be a discrepancy as there are 8 unique location types, which might be due to inconsistent labeling.

**Outlet_Type:** There are 4 unique outlet types, with "Supermarket Type1" being the most frequent.

**Item_Outlet_Sales:** The sales range from 33.29 to 13,086.96, with an average of about 2,181.29.

**Profit:** The profit percentage ranges from 0.1% to 24%, with an average of around 13.41%.

Next, let's visualize the distribution of numerical and categorical features.

### EDA of numerical features

In [ ]:

numeric_features = bigmart_data.select_dtypes(include=[np.number])

# Histogram for all numeric features
numeric_features.hist(figsize=(15, 10), bins=50)
plt.tight_layout()
plt.show()


The histograms above show the distribution of the numerical features in the dataset. Here's a brief summary:

**Item_MRP:** The maximum retail price seems to have multiple peaks, indicating different pricing categories or clusters.
    
**Item_Outlet_Sales:** The sales distribution is right-skewed, with a majority of items having lower sales.
    
**Item_Visibility:** It's also right-skewed, indicating that many items have low visibility in the store.
    
**Item_Weight:** Appears somewhat uniformly distributed, with slight variations.
    
**Outlet_Establishment_Year:** The distribution shows peaks corresponding to specific years, indicating when most outlets were established.
    
**Profit:** The profit percentage appears left-skewed with most items having a profit percentage between 10% and 20%.

In [ ]:
# Selecting numeric features excluding the target 'Item_Outlet_Sales'
features = bigmart_data.select_dtypes(include=[np.number]).drop(columns=['Item_Outlet_Sales'])

# Plotting scatter plots for each numeric feature against 'Item_Outlet_Sales'
for feature in features.columns:
    plt.figure(figsize=(10, 6))
    plt.scatter(bigmart_data[feature], bigmart_data['Item_Outlet_Sales'], alpha=0.5)
    plt.title(f'Relationship between {feature} and Item_Outlet_Sales')
    plt.xlabel(feature)
    plt.ylabel('Item_Outlet_Sales')
    plt.grid(True)
    plt.show()

### EDA of categorical features

In [ ]:

categorical_features = bigmart_data.select_dtypes(exclude=[np.number])

# Countplot for all categorical features
for feature in categorical_features.columns:
    plt.figure(figsize=(12, 5))
    sns.countplot(data=bigmart_data, x=feature)
    plt.title(f'Distribution of {feature}')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


In [ ]:
# Selecting categorical features
categorical_features = bigmart_data.select_dtypes(exclude=[np.number])

# Plotting bar charts for each categorical feature
for feature in categorical_features.columns:
    plt.figure(figsize=(12, 6))
    bigmart_data[feature].value_counts().plot(kind='bar')
    plt.title(f'Distribution of {feature}')
    plt.ylabel('Frequency')
    plt.xlabel(feature)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# Selecting categorical features
categorical_features = bigmart_data.select_dtypes(exclude=[np.number])

# Plotting box plots for each categorical feature against 'Item_Outlet_Sales'
for feature in categorical_features.columns:
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=bigmart_data, x=feature, y='Item_Outlet_Sales')
    plt.title(f'Impact of {feature} on Item_Outlet_Sales')
    plt.ylabel('Item_Outlet_Sales')
    plt.xlabel(feature)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


## 2. Deal with Missings

Identify columns with missing values and decide on strategies to handle them.

In [ ]:
# Check for missing values
missing_values = bigmart_data.isnull().sum()
missing_values[missing_values > 0]

### For imputation, we have several strategies:

**Item_Weight:** We can replace missing values with the mean or median of the Item_Weight column.

**Outlet_Size:** Since it's a categorical variable, we can replace missing values with the mode (most frequent category) of the Outlet_Size column.

**Outlet_Location_Type:** Similarly, being a categorical variable, we can use the mode for imputation.

In [ ]:
# Imputation for 'Item_Weight' using the mean
bigmart_data['Item_Weight'].fillna(bigmart_data['Item_Weight'].mean(), inplace=True)

# Imputation for 'Outlet_Size' using the mode
bigmart_data['Outlet_Size'].fillna(bigmart_data['Outlet_Size'].mode()[0], inplace=True)

# Imputation for 'Outlet_Location_Type' using the mode
bigmart_data['Outlet_Location_Type'].fillna(bigmart_data['Outlet_Location_Type'].mode()[0], inplace=True)

# Verify if missing values are handled
missing_values_after = bigmart_data.isnull().sum()
missing_columns_after = missing_values_after[missing_values_after > 0]
print(missing_columns_after)

# Duplicates and Outliers

### Duplicates
Identify and count duplicate rows.

Remove duplicate rows.

In [ ]:
# Identify and count duplicate rows
duplicate_rows = bigmart_data.duplicated().sum()
print(f'Number of duplicate rows: {duplicate_rows}')

# Remove duplicate rows
bigmart_data.drop_duplicates(inplace=True)

# Verify removal of duplicates
duplicate_rows_after = bigmart_data.duplicated().sum()
print(f'Number of duplicate rows after removal: {duplicate_rows_after}')

### Outliers

Visualize the distribution of numerical features using box plots to identify potential outliers.

Decide on a strategy to handle outliers (e.g., removal, capping, or transformation).

In [ ]:
# Selecting numeric features from the dataset
numeric_features = bigmart_data.select_dtypes(include=[np.number])

# Plotting box plots for each numeric feature to visualize potential outliers
for feature in numeric_features.columns:
    plt.figure(figsize=(10, 6))
    sns.boxplot(bigmart_data[feature])
    plt.title(f'Box plot of {feature}')
    plt.xlabel(feature)
    plt.tight_layout()
    plt.show()

After visualizing the outliers, the next steps typically involve deciding how to handle them. Common strategies include:

**Removal:** This involves simply removing the outlier data points. It's a direct method but can lead to a loss of data.

**Capping:** Outliers are set to a particular maximum or minimum value. This can be set based on domain knowledge or specific percentiles.

**Transformation:** Applying transformations like log, square root, or others to compress the extreme values.
**Binning:** Convert continuous variables into categorical counterparts by creating intervals.

To proceed, we first need to define what we consider an "outlier." A common method is to use the IQR (Interquartile Range) rule:

Any value below Q1−1.5×IQR or above Q3+1.5×IQR is considered an outlier, where Q1 and Q3 are the first and third quartiles, respectively.

In [ ]:
# Function to detect outliers based on IQR
def detect_outliers(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    outliers = ((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR)))
    return outliers

outliers_detected = numeric_features.apply(detect_outliers)
outliers_count = outliers_detected.sum()
print(outliers_count)

Transformations can be an effective way to manage outliers, especially for data that is skewed. Common transformations include:

**Log Transformation:** Useful for right-skewed data.
    
**Square Root Transformation:** Helps to reduce the impact of extreme values.
    
**Box-Cox Transformation:** A family of power transformations that can stabilize variance and make the data more normal in distribution.

Given the nature of sales data and the usual presence of zero values in features like Item_Visibility, the log transformation might be tricky due to the logarithm of zero being undefined. Therefore, we can use a slight adjustment by adding a small constant to ensure there are no zero values.

Let's start by visualizing the distribution of these features using histograms to determine their skewness and then decide on an appropriate transformation.

In [ ]:
features_to_transform = ['Item_Visibility', 'Item_Outlet_Sales', 'Profit']

# Plotting histograms for the specified features
for feature in features_to_transform:
    plt.figure(figsize=(10, 6))
    sns.histplot(bigmart_data[feature], kde=True)
    plt.title(f'Distribution of {feature}')
    plt.xlabel(feature)
    plt.tight_layout()
    plt.show()

The Box-Cox transformation is indeed a powerful method to handle skewed data. It's defined as:

$$
\begin{equation}
y_{\text{transformed}} =
\begin{cases}
\frac{y^\lambda - 1}{\lambda}, & \text{if } \lambda \neq 0 \\
\ln(y), & \text{if } \lambda = 0
\end{cases}
\end{equation}
$$

Where:

Y is the data you want to transform.

λ is the transformation parameter.

The Box-Cox transformation requires all data to be positive. So before applying it, we'll ensure the data is positive. Also, we'll use the scipy library's boxcox function, which automatically finds the lambda that maximizes the normality of the data.

Here's the Python code to apply the Box-Cox transformation to the specified features:

In [ ]:
features_to_transform = ['Item_Visibility', 'Item_Outlet_Sales']

# Ensure data is positive by adding a small constant to 'Item_Visibility'
bigmart_data['Item_Visibility'] = bigmart_data['Item_Visibility'] + 0.01

# Applying Box-Cox transformation
for feature in features_to_transform:
    bigmart_data[feature], _ = boxcox(bigmart_data[feature])


for feature in features_to_transform:
    plt.figure(figsize=(10, 6))
    sns.histplot(bigmart_data[feature], kde=True)
    plt.title(f'Distribution of {feature} after Box-Cox Transformation')
    plt.xlabel(feature)
    plt.tight_layout()
    plt.show()


In [ ]:
# Ensure all values are positive by adding a small constant to 'Profit'
bigmart_data['Profit'] = bigmart_data['Profit'] + abs(bigmart_data['Profit'].min()) + 0.01

# Applying the log transformation
bigmart_data['Profit_log_transformed'] = np.log(bigmart_data['Profit'])

# Visualizing the transformed data
plt.figure(figsize=(10, 6))
sns.histplot(bigmart_data['Profit_log_transformed'], kde=True)
plt.title('Distribution of Profit after Log Transformation')
plt.xlabel('Profit (Log Transformed)')
plt.tight_layout()
plt.show()

The log transformation may not always produce a perfectly normal distribution, especially if the original data is highly skewed or has a complex distribution.

Another transformation you might consider is the inverse transformation, especially when the data is skewed to the right. For data skewed to the left, a square or cube transformation might be effective.

Let's try the inverse transformation on the Profit feature:

Here's the Python code to apply the inverse transformation to the Profit feature:

In [ ]:
# Applying the inverse transformation to the 'Profit' feature
bigmart_data['Profit_inverse_transformed'] = 1 / bigmart_data['Profit']

# Visualizing the transformed data
plt.figure(figsize=(10, 6))
sns.histplot(bigmart_data['Profit_inverse_transformed'], kde=True)
plt.title('Distribution of Profit after Inverse Transformation')
plt.xlabel('Profit (Inverse Transformed)')
plt.tight_layout()
plt.show()

If the Profit feature has a complicated distribution, common transformations like log and inverse might not produce a perfectly normal distribution. Let's explore a few more alternatives:

**Square Transformation:** For left-skewed data.

**Sqrt Transformation:** For right-skewed data.

**Yeo-Johnson Transformation:** A more flexible method that can handle zeros in the data.

**Quantile Transformer:** Maps the data to a uniform distribution.

Given the right skew, we can try the Sqrt transformation and the Yeo-Johnson transformation. The Quantile Transformer is also worth considering, as it makes fewer assumptions about the functional form of the transformation.

Let's start with the Sqrt transformation on the Profit feature:

Here's the Python code to apply the Sqrt transformation to the Profit feature:

In [ ]:
# Applying the square root transformation to the 'Profit' feature
bigmart_data['Profit_sqrt_transformed'] = np.sqrt(bigmart_data['Profit'])

# Visualizing the transformed data
plt.figure(figsize=(10, 6))
sns.histplot(bigmart_data['Profit_sqrt_transformed'], kde=True)
plt.title('Distribution of Profit after Square Root Transformation')
plt.xlabel('Profit (Square Root Transformed)')
plt.tight_layout()
plt.show()

The Yeo-Johnson transformation is a flexible method that can handle both positive and negative values in the data, making it a generalization of the Box-Cox transformation. It's particularly useful when data has zeros or negative values.

Let's apply the Yeo-Johnson transformation to the Profit feature.

Here's the Python code to apply the Yeo-Johnson transformation:

In [ ]:
# Applying the Yeo-Johnson transformation to the 'Profit' feature
bigmart_data['Profit_yeojohnson_transformed'], lambda_value = yeojohnson(bigmart_data['Profit'])

# Visualizing the transformed data
plt.figure(figsize=(10, 6))
sns.histplot(bigmart_data['Profit_yeojohnson_transformed'], kde=True)
plt.title('Distribution of Profit after Yeo-Johnson Transformation')
plt.xlabel('Profit (Yeo-Johnson Transformed)')
plt.tight_layout()
plt.show()

The Quantile Transformer is a robust method that can map data to a uniform distribution (or a Gaussian distribution) by ranking the data. This can be particularly useful for dealing with features that have heavy tails or features that are on different scales.

The QuantileTransformer class in the sklearn.preprocessing module can be used for this transformation.

Here's the Python code to apply the Quantile Transformer to the Profit feature:

In [ ]:
# Applying the Quantile Transformer to the 'Profit' feature
transformer = QuantileTransformer(output_distribution='uniform', random_state=0)
bigmart_data['Profit_quantile_transformed'] = transformer.fit_transform(bigmart_data[['Profit']])

# Visualizing the transformed data
plt.figure(figsize=(10, 6))
sns.histplot(bigmart_data['Profit_quantile_transformed'], kde=True)
plt.title('Distribution of Profit after Quantile Transformation')
plt.xlabel('Profit (Quantile Transformed)')
plt.tight_layout()
plt.show()

# Feature Encoding

Feature encoding is essential when dealing with categorical variables, especially for machine learning algorithms that expect numerical input values.

There are several methods for encoding categorical variables. Some of the most common ones include:

**One-Hot Encoding:** This method creates a binary column for each category in the original column. It's suitable for nominal data where no ordinal relationship exists.
    
**Label Encoding:** This method assigns a unique integer to each category. It's more suitable for ordinal data where an order between categories exists, but can be used for nominal data with tree-based algorithms.
    
**Ordinal Encoding:** Similar to label encoding but used explicitly for ordinal data where an order between categories exists.
    
**Frequency or Count Encoding:** Replace categories by the count of the occurrences or the frequency (proportion).

**Target Encoding (Mean Encoding):** Replace categories by the average target value for that category. Care must be taken to avoid data leakage.

For the Bigmart dataset, there are several categorical variables. We'll start by identifying them and then decide on the appropriate encoding technique for each. Let's first list out the categorical features in the dataset.

In [ ]:
# Identifying categorical features in the dataset
categorical_features = bigmart_data.select_dtypes(include=['object']).columns
print(categorical_features)

Let's examine each of the categorical features in the dataset and provide recommendations based on their nature:

**Item_Identifier:** This is an identification code for the product. As it likely has a large number of unique values, one-hot encoding might significantly increase the dimensionality of the dataset. A better approach might be to drop this feature unless it's essential for specific analyses or to use feature engineering to derive useful information from it.

**Item_Fat_Content:** This is nominal data representing the fat content of the item. One-hot encoding would be appropriate here. Additionally, this column might have inconsistent labeling (e.g., "Low Fat", "LF" both indicating low fat), so it would be good to standardize the labels first.

**Item_Type:** This is the type or category of the item, which is nominal data. One-hot encoding is recommended.

**Outlet_Identifier:** Like Item_Identifier, this is an identification code for the outlet. If there's a business reason to retain this feature, one-hot encoding might be used. Alternatively, it can be dropped or used to derive other features.

**Outlet_Establishment_Year:** This feature is ordinal since there's a clear order (older or newer year). While it's already in a numeric format, it might be more useful to convert this to the age of the outlet (e.g., "2023 - Outlet_Establishment_Year") to give a clearer picture of how old each outlet is.

**Outlet_Size:** This represents the size of the outlet and is ordinal data (e.g., Small < Medium < Large). Ordinal encoding can be used here.

**Outlet_Location_Type:** This is ordinal data representing the type of city in which the store is located. Ordinal encoding is suitable.

**Outlet_Type:** This is the type of outlet, which is nominal data. One-hot encoding is recommended.

Recommendations:

**Item_Identifier:** Drop or feature engineering.

**Item_Fat_Content:** One-hot encoding (after standardizing labels).

**Item_Type:** One-hot encoding.

**Outlet_Identifier:** Drop or one-hot encoding.

**Outlet_Establishment_Year:** Convert to outlet age.

**Outlet_Size:** Ordinal encoding.

**Outlet_Location_Type:** Ordinal encoding.

**Outlet_Type:** One-hot encoding.

In [ ]:
# Dropping 'Item_Identifier' and 'Outlet_Identifier'
bigmart_data.drop(['Item_Identifier', 'Outlet_Identifier'], axis=1, inplace=True)

# Standardizing labels in 'Item_Fat_Content'
fat_content_mapping = {
    'Low Fat': 'Low Fat', 
    'Regular': 'Regular', 
    'LF': 'Low Fat', 
    'reg': 'Regular', 
    'low fat': 'Low Fat'
}
bigmart_data['Item_Fat_Content'] = bigmart_data['Item_Fat_Content'].map(fat_content_mapping)

# One-hot encoding for 'Item_Fat_Content', 'Item_Type', and 'Outlet_Type'
one_hot_encoder = OneHotEncoder(drop='first', sparse=False)
one_hot_encoded_columns = ['Item_Fat_Content', 'Item_Type', 'Outlet_Type']
one_hot_encoded_data = one_hot_encoder.fit_transform(bigmart_data[one_hot_encoded_columns])
one_hot_encoded_df = pd.DataFrame(one_hot_encoded_data, columns=one_hot_encoder.get_feature_names_out(one_hot_encoded_columns))
bigmart_data = pd.concat([bigmart_data, one_hot_encoded_df], axis=1)
bigmart_data.drop(one_hot_encoded_columns, axis=1, inplace=True)

# Convert 'Outlet_Establishment_Year' to outlet age
current_year = 2023
bigmart_data['Outlet_Age'] = current_year - bigmart_data['Outlet_Establishment_Year']
bigmart_data.drop('Outlet_Establishment_Year', axis=1, inplace=True)

# Ordinal encoding for 'Outlet_Size' and 'Outlet_Location_Type'
ordinal_encodings = {
    'Outlet_Size': {'Small': 1, 'Medium': 2, 'High': 3},
    'Outlet_Location_Type': {'Tier 3': 3, 'Tier 2': 2, 'Tier 1': 1}
}
bigmart_data.replace(ordinal_encodings, inplace=True)

bigmart_data.head()

# Feature Scaling

Feature scaling ensures that all features contribute equally to the computation of distances. There are two common methods:

**Standardization (Z-score normalization):** This method scales the features such that they have the properties of a standard normal distribution with a mean of 0 and a standard deviation of 1.

Formula:

$$
\begin{equation}
z = \frac{x - \mu}{\sigma}
\end{equation}
$$
 
Where:
x = original feature value
μ = mean of the feature
σ = standard deviation of the feature

**Min-Max Scaling:** This method scales and translates each feature individually such that it is in the given range on the training set, typically between zero and one.

Formula:
$$
\begin{equation}
x_{\text{scaled}} = \frac{x - \min(X)}{\max(X) - \min(X)}
\end{equation}
$$
 
Most machine learning algorithms perform better when numerical input variables are scaled to a standard range.

Let's proceed with Min-Max Scaling.

Min-Max Scaling scales features to lie between a given minimum and maximum value, often between 0 and 1. This transformation preserves the shape of the original distribution and doesn't change the information embedded in the original data.

Here's the Python code to apply Min-Max Scaling to the numerical features of the dataset:

In [ ]:
# Identifying numerical columns
numerical_features = bigmart_data.select_dtypes(include=[float, int]).columns

# Applying Min-Max Scaling
scaler = MinMaxScaler()
bigmart_data[numerical_features] = scaler.fit_transform(bigmart_data[numerical_features])

bigmart_data.head()


# Feature engineering

Feature engineering is the process of creating new features or modifying existing ones to improve model performance. It involves using domain knowledge and mathematical techniques to generate features that make the machine learning algorithm work more effectively.

Let's explore some common feature engineering techniques and potential new features we could create for the Bigmart dataset:

**Polynomial Features:** Creating interaction terms or powers of features. This can help in capturing non-linear relationships.

**Binning:** Converting continuous variables into categorical ones by grouping values into bins or intervals.

**Aggregation:** Aggregating data can help in generating higher-level features. For example, we could compute the average sales for each type of item or each outlet.

**Domain-Specific Features:** Using domain knowledge to create new features. For example:

Creating a feature to indicate whether an item is a snack based on its Item_Type.
Creating a feature indicating if an outlet is relatively new or old based on its age.

**Feature Decomposition:** Techniques like PCA (Principal Component Analysis) can be used to reduce dimensionality and create new features.

**Time-based Features:** If we had timestamped data, we could create features like "day of the week," "hour of the day," etc.

Given the Bigmart dataset, here are some potential feature engineering steps we could consider:

**IsSnack:** A binary feature indicating whether an item is a snack or not based on its type.

**Average Sales by Item Type:** Calculate the average sales for each item type.

**Outlet Age Group:** Categorize outlets as "New," "Medium Age," or "Old" based on the Outlet_Age feature.


In [ ]:
# 1. IsSnack: Using the one-hot encoded columns
bigmart_data['IsSnack'] = bigmart_data.apply(lambda row: 1 if row['Item_Type_Breads'] == 1 or row['Item_Type_Snack Foods'] == 1 else 0, axis=1)

# 2. Average Sales by Item Type: We will use the original 'Item_Outlet_Sales' column to calculate this.
average_sales = bigmart_data['Item_Outlet_Sales'].mean()
bigmart_data['Avg_Sales_by_Item_Type'] = average_sales

# 3. Outlet Age Group: Use the 'Outlet_Age' feature derived earlier.
bins = [0, 10, 20, 35]  # Bins for new (0-10), medium age (10-20), and old (20-35)
labels = ['New', 'Medium Age', 'Old']
bigmart_data['Outlet_Age_Group'] = pd.cut(bigmart_data['Outlet_Age'], bins=bins, labels=labels, right=True)

bigmart_data[['IsSnack', 'Avg_Sales_by_Item_Type', 'Outlet_Age_Group']].head()

# Feature selection

Feature selection is the process of reducing the number of input variables when developing a predictive model. Proper feature selection can lead to simpler, more interpretable models that perform better on new data.

There are various methods for feature selection:

**Filter Methods:** These are based on the general characteristics of data. They use statistical measures to score each feature. Common techniques include:

**Correlation Coefficient:** Removing features that are highly correlated.

**Chi-Squared Test:** Used for categorical data to test the relationship between each feature and the target.

**Wrapper Methods:** These use a subset of features and train a model using them. Based on the inferences drawn from the previous model, they decide to add or remove features. 

Common techniques include:

**Recursive Feature Elimination (RFE):** A type of wrapper method.

**Embedded Methods:** These are iterative in a sense that takes care of each iteration of the model training process and carefully extracts those features that contribute the most to the training for a particular iteration. Common techniques include:

**LASSO Regression:** Adds penalty equivalent to the absolute value of the magnitude of coefficients.

**Tree-based Methods:** Like Decision Trees, Random Forests, and Gradient Boosted Trees can be used to rank feature importance.

For the Bigmart dataset and considering its size and the number of features, I'd recommend starting with:

**Correlation Coefficient:** To remove features that are highly correlated.

**Tree-based Methods:** Using a tree-based method like a Random Forest to rank and select the most important features.


In [ ]:
# Compute the correlation matrix
correlation_matrix = bigmart_data.corr()

# Plot the heatmap
plt.figure(figsize=(16, 12))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Correlation Matrix")
plt.show()

In [ ]:
# Finding the columns and the number of occurrences of ' --'
occurrences = bigmart_data[bigmart_data == '  --'].count()
occurrences = occurrences[occurrences > 0]
occurrences

The value ' --' in Outlet_Location_Type indicates missing or unrecorded data. We have a few options to handle this:

**Remove Rows:** If the number of rows with ' --' is small relative to the dataset's size, we could remove these rows.
**Impute with Mode:** Replace ' --' with the most common value (mode) in the Outlet_Location_Type column. This is a typical method for handling missing categorical data.
**Create a New Category:** Treat ' --' as a separate category. This might be useful if ' --' has a special meaning, but it can introduce noise if it's merely a placeholder for missing data.

Given our previous discussions and data preprocessing steps, I'd recommend imputing with the mode. 


In [ ]:
# Impute ' --' with the mode in the 'Outlet_Location_Type' column
mode_value = bigmart_data['Outlet_Location_Type'].mode()[0]
bigmart_data['Outlet_Location_Type'].replace('  --', mode_value, inplace=True)

# Checking if ' --' has been replaced
remaining_occurrences = bigmart_data[bigmart_data == '  --'].count().sum()
remaining_occurrences


After examining the correlation matrix and removing any features that are highly correlated, we will use the Random Forest algorithm to rank and select the most important features.

## Tree-based Method: Random Forest
Random Forest is an ensemble learning method that can be used for both classification and regression tasks. It works by constructing multiple decision trees during training and outputs the class that is the mode of the classes (classification) or the mean prediction (regression) of the individual trees.

Let's proceed with using a Random Forest to rank the importance of features.

In [ ]:
# One-hot encoding for 'Outlet_Location_Type'
one_hot_encoded_data = pd.get_dummies(bigmart_data['Outlet_Location_Type'], drop_first=True, prefix='Outlet_Location_Type')
bigmart_data = pd.concat([bigmart_data, one_hot_encoded_data], axis=1)
bigmart_data.drop('Outlet_Location_Type', axis=1, inplace=True)

# One-hot encoding for 'Outlet_Age_Group'
one_hot_encoded_age_group = pd.get_dummies(bigmart_data['Outlet_Age_Group'], drop_first=True, prefix='Outlet_Age_Group')
bigmart_data = pd.concat([bigmart_data, one_hot_encoded_age_group], axis=1)
bigmart_data.drop('Outlet_Age_Group', axis=1, inplace=True)

# Separating features and target variable
X = bigmart_data.drop('Item_Outlet_Sales', axis=1)
y = bigmart_data['Item_Outlet_Sales']

# Initializing and fitting the Random Forest model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X, y)

# Getting feature importances
feature_importances = pd.Series(rf.feature_importances_, index=X.columns)

# Sorting feature importances in descending order and plotting
feature_importances_sorted = feature_importances.sort_values(ascending=False)
feature_importances_sorted.plot(kind='barh', figsize=(12, 10), color='skyblue')
plt.title('Feature Importances using Random Forest')
plt.show()

Based on the feature importance plot generated by the Random Forest model, we can make a decision on which features to discard.

Typically, features with the least importance (those at the bottom of the plot) contribute the least to the model's predictive power and can be considered for removal.

Here are the steps to discard less important features:

Set a threshold: Decide on a threshold below which feature importance is considered low.

List features below the threshold: Identify features whose importance is below the set threshold.

Remove those features from the dataset.

Let's determine which features fall below a certain threshold of importance. For demonstration purposes, I'll set a threshold that captures features with an importance below 0.01, but this can be adjusted based on your preferences.

In [ ]:
# Separate features and target
X = bigmart_data.drop('Item_Outlet_Sales', axis=1)
y = bigmart_data['Item_Outlet_Sales']

# Train Random Forest and get feature importances
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X, y)
feature_importances = pd.Series(rf.feature_importances_, index=X.columns)

# Discard features with importance below 0.01
features_to_discard = feature_importances[feature_importances < 0.01].index.tolist()
bigmart_data_reduced = bigmart_data.drop(features_to_discard, axis=1)

# Display the columns that were removed
features_to_discard

# Data Splitting

The next step in the pipeline is to split the dataset into a training set and a testing set. This allows us to train a machine learning model on one subset and validate its performance on another, unseen subset.

Typically, we use about 70-80% of the data for training and the remaining 20-30% for testing. However, this ratio can vary based on the dataset size and the specific problem at hand.

I'll proceed to split the data into 80% for training and 20% for testing. If you have a different preference for the split ratio or any other specifics

In [ ]:
# Split the dataset
X = bigmart_data_reduced.drop('Item_Outlet_Sales', axis=1)
y = bigmart_data['Item_Outlet_Sales']

# Splitting the data (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape

# Model Building

For model building, we have a plethora of algorithms to choose from. Given the nature of the dataset and the task (predicting sales, which is a regression problem), here are some commonly used regression algorithms:

**Linear Regression:** A simple linear approach to establish a relationship between the dependent and independent variables. It's a good starting point.

**Decision Trees and Random Forest:** Trees partition the data space into regions, and Random Forest is an ensemble of such trees. They can capture non-linear relationships.

**Gradient Boosted Trees (e.g., XGBoost, LightGBM):** These are boosting algorithms that build trees in a sequential manner, where each tree tries to correct the errors of the previous one.

**Support Vector Regression:** Uses the support vector machine (SVM) algorithm for regression.

**Neural Networks:** Multi-layer perceptrons can be used for regression tasks and can capture complex, non-linear relationships.

As a starting point, I'd recommend beginning with a simple Linear Regression to establish a baseline, then trying more complex models like Random Forest or Gradient Boosted Trees to see if we can improve performance.



## Linear Regression

let's proceed with building a Linear Regression model.

Here are the steps:

Initialize the Linear Regression model.

Train (fit) the model using the training data.

Evaluate the model's performance on the training data and the testing data using metrics like Mean Absolute Error (MAE), Mean Squared Error (MSE), and R-squared.

In [ ]:
# Split the dataset
X = bigmart_data_reduced.drop('Item_Outlet_Sales', axis=1)
y = bigmart_data['Item_Outlet_Sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Predictions
train_predictions = lr_model.predict(X_train)
test_predictions = lr_model.predict(X_test)

# Evaluate the model's performance
mae_train = mean_absolute_error(y_train, train_predictions)
mse_train = mean_squared_error(y_train, train_predictions)
r2_train = r2_score(y_train, train_predictions)

mae_test = mean_absolute_error(y_test, test_predictions)
mse_test = mean_squared_error(y_test, test_predictions)
r2_test = r2_score(y_test, test_predictions)

print("Training Metrics:")
print(f"Mean Absolute Error (MAE): {mae_train}")
print(f"Mean Squared Error (MSE): {mse_train}")
print(f"R-squared: {r2_train}\n")

print("Testing Metrics:")
print(f"Mean Absolute Error (MAE): {mae_test}")
print(f"Mean Squared Error (MSE): {mse_test}")
print(f"R-squared: {r2_test}")

After executing the code, you'll get the performance metrics of the Linear Regression model for both the training and testing datasets.

This will provide you with a sense of how well the model fits the training data and how well it generalizes to new, unseen data. The R-squared value will give an indication of the proportion of the variance in the dependent variable that is predictable from the independent variables.

Once you've done this, you can decide if you want to explore more complex models or tune the current one further. 

## Random Forest

Let's proceed with building a Random Forest model.

The steps involved are:

Initialize the Random Forest Regressor.

Train (fit) the model using the training data.

Evaluate the model's performance on the training data and the testing data using metrics like Mean Absolute Error (MAE), Mean Squared Error (MSE), and R-squared.

Let me provide you with the code to execute these steps:

In [ ]:
# 1. Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# 2. Train the model
rf_model.fit(X_train, y_train)

# 3. Predictions
rf_train_predictions = rf_model.predict(X_train)
rf_test_predictions = rf_model.predict(X_test)

# Evaluate the model's performance
rf_mae_train = mean_absolute_error(y_train, rf_train_predictions)
rf_mse_train = mean_squared_error(y_train, rf_train_predictions)
rf_r2_train = r2_score(y_train, rf_train_predictions)

rf_mae_test = mean_absolute_error(y_test, rf_test_predictions)
rf_mse_test = mean_squared_error(y_test, rf_test_predictions)
rf_r2_test = r2_score(y_test, rf_test_predictions)

print("Training Metrics:")
print(f"Mean Absolute Error (MAE): {rf_mae_train}")
print(f"Mean Squared Error (MSE): {rf_mse_train}")
print(f"R-squared: {rf_r2_train}\n")

print("Testing Metrics:")
print(f"Mean Absolute Error (MAE): {rf_mae_test}")
print(f"Mean Squared Error (MSE): {rf_mse_test}")
print(f"R-squared: {rf_r2_test}")

After executing the code, you'll get the performance metrics of the Random Forest model for both the training and testing datasets. This will help you understand the model's fit to the training data and its generalization to new data.

Once you have the results, you can compare the performance with the Linear Regression model. If Random Forest offers significant improvement, you might consider fine-tuning its hyperparameters for even better results.

## XGBoost

Alright, if Random Forest didn't provide a significant improvement, the next model I'd recommend trying is the Gradient Boosted Trees model, specifically using XGBoost.

XGBoost is a popular gradient boosting library that often performs well in machine learning competitions and real-world datasets.

Here's how you can train and evaluate an XGBoost regressor on your dataset:

Install the XGBoost library (if you haven't already).

Initialize the XGBoost regressor.

Train (fit) the model using the training data.

Evaluate the model's performance using the same metrics as before.

Let's proceed with these steps.

To train and evaluate an XGBoost regressor, follow these steps:

In [ ]:
# Initialize the XGBoost regressor
xgb_model = xgb.XGBRegressor(objective ='reg:squarederror', n_estimators=100, random_state=42)

# Train the model
xgb_model.fit(X_train, y_train)

# Predictions
xgb_train_predictions = xgb_model.predict(X_train)
xgb_test_predictions = xgb_model.predict(X_test)

# Evaluate the model's performance
xgb_mae_train = mean_absolute_error(y_train, xgb_train_predictions)
xgb_mse_train = mean_squared_error(y_train, xgb_train_predictions)
xgb_r2_train = r2_score(y_train, xgb_train_predictions)

xgb_mae_test = mean_absolute_error(y_test, xgb_test_predictions)
xgb_mse_test = mean_squared_error(y_test, xgb_test_predictions)
xgb_r2_test = r2_score(y_test, xgb_test_predictions)

print("Training Metrics:")
print(f"Mean Absolute Error (MAE): {xgb_mae_train}")
print(f"Mean Squared Error (MSE): {xgb_mse_train}")
print(f"R-squared: {xgb_r2_train}\n")

print("Testing Metrics:")
print(f"Mean Absolute Error (MAE): {xgb_mae_test}")
print(f"Mean Squared Error (MSE): {xgb_mse_test}")
print(f"R-squared: {xgb_r2_test}")

If the training performance is good but the testing performance is lagging, it indicates that the model might be overfitting. Overfitting happens when the model learns the training data too closely, including its noise and outliers, and thus performs poorly on unseen data.

Here are a few strategies to address overfitting in XGBoost:

**Increase Regularization:** XGBoost provides parameters like alpha (L1 regularization) and lambda (L2 regularization) that can be adjusted to add regularization to the model.

**Reduce Model Complexity:** Reduce the depth of the trees by setting a lower value for the max_depth parameter.

**Add Early Stopping:** Train the model for a large number of rounds and use early stopping to stop training once the model's performance on a validation set stops improving.

**Reduce Learning Rate:** A smaller learning rate with a higher number of boosting rounds can sometimes help in getting a more generalized model.

**Feature Selection:** Ensure that only relevant features are being used. Irrelevant or redundant features can lead to overfitting.

Let's start by implementing some of these strategies with XGBoost.

Alright, let's start by implementing a combination of the strategies mentioned to combat overfitting in the XGBoost model:

**Reduce Learning Rate:** We'll set a smaller learning rate and increase the number of boosting rounds.

**Add Early Stopping:** We'll use early stopping with a validation set to prevent overfitting.

**Increase Regularization:** We'll add L1 and L2 regularization to the model.

In [ ]:
# Splitting the training data further into training and validation sets for early stopping
X_train_sub, X_val, y_train_sub, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


# Initialize the XGBoost regressor with adjusted parameters
xgb_model_adjusted = xgb.XGBRegressor(
    objective ='reg:squarederror', 
    n_estimators=1000,  # Increase boosting rounds
    learning_rate=0.01,  # Reduce learning rate
    reg_alpha=10,  # L1 regularization
    reg_lambda=10,  # L2 regularization
    max_depth=4,  # Reduce model complexity
    random_state=42
)

# Train the model with early stopping
xgb_model_adjusted.fit(
    X_train_sub, y_train_sub, 
    early_stopping_rounds=10, 
    eval_set=[(X_val, y_val)], 
    verbose=False
)

# Predictions
xgb_train_predictions_adjusted = xgb_model_adjusted.predict(X_train)
xgb_test_predictions_adjusted = xgb_model_adjusted.predict(X_test)

# Evaluate the adjusted model's performance
xgb_mae_train_adjusted = mean_absolute_error(y_train, xgb_train_predictions_adjusted)
xgb_mse_train_adjusted = mean_squared_error(y_train, xgb_train_predictions_adjusted)
xgb_r2_train_adjusted = r2_score(y_train, xgb_train_predictions_adjusted)

xgb_mae_test_adjusted = mean_absolute_error(y_test, xgb_test_predictions_adjusted)
xgb_mse_test_adjusted = mean_squared_error(y_test, xgb_test_predictions_adjusted)
xgb_r2_test_adjusted = r2_score(y_test, xgb_test_predictions_adjusted)

print("Training Metrics:")
print(f"Mean Absolute Error (MAE): {xgb_mae_train_adjusted}")
print(f"Mean Squared Error (MSE): {xgb_mse_train_adjusted}")
print(f"R-squared: {xgb_r2_train_adjusted}\n")

print("Testing Metrics:")
print(f"Mean Absolute Error (MAE): {xgb_mae_test_adjusted}")
print(f"Mean Squared Error (MSE): {xgb_mse_test_adjusted}")
print(f"R-squared: {xgb_r2_test_adjusted}")

Running the above code will train an XGBoost model with the suggested adjustments and provide performance metrics for both training and testing datasets.

After evaluating the results, we can decide if further hyperparameter tuning is necessary or if we should explore a different model.

The fact that the training and testing scores are now closer suggests that the overfitting issue has been somewhat addressed. However, the performance might still not be satisfactory. Here are some potential next steps:

**Hyperparameter Tuning:** Use tools like GridSearchCV or RandomizedSearchCV from Scikit-learn to systematically search for the best hyperparameters for the XGBoost model.

**Try Different Models:** As previously mentioned, you could explore models like Support Vector Regression (SVR), Neural Networks, or Gradient Boosted Trees using libraries like LightGBM or CatBoost.

**Feature Engineering:** Revisit the features in the dataset. Creating new features or transforming existing ones can sometimes help in improving model performance.

**Data Collection:** If possible, gather more data. A larger dataset might help in improving the model's generalization.

**Model Stacking:** Combine predictions from multiple models to improve performance.


## GridSearchCV

Hyperparameter tuning is a crucial step in improving model performance. We'll use GridSearchCV to systematically search for the best hyperparameters for the XGBoost model.

Here's the plan:

Define a set of hyperparameters to search.

Use GridSearchCV to evaluate all the possible combinations of hyperparameters.

Train the XGBoost model with the best hyperparameters found.

Evaluate the model's performance.


Let's proceed with this approach.

Here's how you can perform hyperparameter tuning using GridSearchCV with XGBoost:

In [ ]:
# Define the hyperparameters and their possible values
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5, 6],
    'min_child_weight': [1, 5, 10],
    'subsample': [0.5, 0.7, 1.0],
    'colsample_bytree': [0.5, 0.7, 1.0],
    'n_estimators': [100, 500, 1000],
    'reg_alpha': [1e-5, 1e-2, 0.1, 1, 100],
    'reg_lambda': [1e-5, 1e-2, 0.1, 1, 100]
}

# Initialize XGBoost regressor
xgb_model_for_gs = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Set up GridSearchCV
grid_search = GridSearchCV(xgb_model_for_gs, param_grid, cv=3, n_jobs=-1, verbose=1, scoring='r2')

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best parameters
best_parameters = grid_search.best_params_
best_estimator = grid_search.best_estimator_

# Predictions using the model with the best parameters
xgb_gs_train_predictions = best_estimator.predict(X_train)
xgb_gs_test_predictions = best_estimator.predict(X_test)

# Evaluate the model's performance
xgb_gs_mae_train = mean_absolute_error(y_train, xgb_gs_train_predictions)
xgb_gs_mse_train = mean_squared_error(y_train, xgb_gs_train_predictions)
xgb_gs_r2_train = r2_score(y_train, xgb_gs_train_predictions)

xgb_gs_mae_test = mean_absolute_error(y_test, xgb_gs_test_predictions)
xgb_gs_mse_test = mean_squared_error(y_test, xgb_gs_test_predictions)
xgb_gs_r2_test = r2_score(y_test, xgb_gs_test_predictions)

print("Best Parameters:", best_parameters)
print("\nTraining Metrics:")
print(f"Mean Absolute Error (MAE): {xgb_gs_mae_train}")
print(f"Mean Squared Error (MSE): {xgb_gs_mse_train}")
print(f"R-squared: {xgb_gs_r2_train}\n")

print("Testing Metrics:")
print(f"Mean Absolute Error (MAE): {xgb_gs_mae_test}")
print(f"Mean Squared Error (MSE): {xgb_gs_mse_test}")
print(f"R-squared: {xgb_gs_r2_test}")

This script will search over the defined hyperparameters using cross-validation. Once the best parameters are identified, the model will be evaluated on the training and testing data.

Please note: Grid search can be computationally intensive, especially with a large grid like the one defined. Depending on your computational resources, this process might take some time.

After executing the script, compare the performance metrics with the previous models to determine if there's an improvement. If the results are still unsatisfactory, further tuning or trying a different model might be necessary.

## LightGBM

LightGBM is another gradient boosting framework that uses tree-based learning algorithms, and it's known for its efficiency and speed. It can handle large datasets and is optimized for performance.

Let's proceed with building a LightGBM model for your dataset:

Install the LightGBM library (if you haven't already).
                              
Initialize the LightGBM regressor.
                              
Train (fit) the model using the training data.
                              
Evaluate the model's performance using metrics like Mean Absolute Error (MAE), Mean Squared Error (MSE), and R-squared.



In [ ]:
# Initialize the LightGBM regressor
lgb_model = lgb.LGBMRegressor(objective='regression', n_estimators=1000, learning_rate=0.01, random_state=42)

# Train the model
lgb_model.fit(X_train, y_train)

# Predictions
lgb_train_predictions = lgb_model.predict(X_train)
lgb_test_predictions = lgb_model.predict(X_test)

# Evaluate the model's performance
lgb_mae_train = mean_absolute_error(y_train, lgb_train_predictions)
lgb_mse_train = mean_squared_error(y_train, lgb_train_predictions)
lgb_r2_train = r2_score(y_train, lgb_train_predictions)

lgb_mae_test = mean_absolute_error(y_test, lgb_test_predictions)
lgb_mse_test = mean_squared_error(y_test, lgb_test_predictions)
lgb_r2_test = r2_score(y_test, lgb_test_predictions)

print("Training Metrics:")
print(f"Mean Absolute Error (MAE): {lgb_mae_train}")
print(f"Mean Squared Error (MSE): {lgb_mse_train}")
print(f"R-squared: {lgb_r2_train}\n")

print("Testing Metrics:")
print(f"Mean Absolute Error (MAE): {lgb_mae_test}")
print(f"Mean Squared Error (MSE): {lgb_mse_test}")
print(f"R-squared: {lgb_r2_test}")

After executing this code, you'll get the performance metrics of the LightGBM model for both the training and testing datasets. If the LightGBM model provides an improvement, further hyperparameter tuning or model adjustments can be explored.

## Model Stacking

Model stacking is a technique where you use the predictions from multiple models to train a meta-model (or blender) that then makes the final prediction. The idea is to take advantage of the strengths of each individual model to produce a better and more generalized prediction.

Here's a simplified approach to model stacking:

Split the training set into two subsets.

Train multiple models on the first subset.

Use the trained models to make predictions on the second subset.

Use the predictions from step 3 as input features to train a meta-model.

Use the meta-model to make predictions on the test set.

For this example, we'll use three base models:

Linear Regression

Random Forest Regressor

XGBoost Regressor

And our meta-model will be a simple Linear Regression.

Let's proceed with this approach.

Here's how you can implement model stacking:

In [ ]:
X_train_sub1, X_train_sub2, y_train_sub1, y_train_sub2 = train_test_split(X_train, y_train, test_size=0.5, random_state=42)

In [ ]:
# Initialize models
lr = LinearRegression()
rf = RandomForestRegressor(n_estimators=100, random_state=42)
xg = xgb.XGBRegressor(objective ='reg:squarederror', n_estimators=100, random_state=42)

# Train models
lr.fit(X_train_sub1, y_train_sub1)
rf.fit(X_train_sub1, y_train_sub1)
xg.fit(X_train_sub1, y_train_sub1)

lr_pred = lr.predict(X_train_sub2)
rf_pred = rf.predict(X_train_sub2)
xg_pred = xg.predict(X_train_sub2)

stacked_predictions = np.column_stack((lr_pred, rf_pred, xg_pred))

# Train meta-model
meta_model = LinearRegression()
meta_model.fit(stacked_predictions, y_train_sub2)

lr_test_pred = lr.predict(X_test)
rf_test_pred = rf.predict(X_test)
xg_test_pred = xg.predict(X_test)

stacked_test_predictions = np.column_stack((lr_test_pred, rf_test_pred, xg_test_pred))
final_predictions = meta_model.predict(stacked_test_predictions)


mae = mean_absolute_error(y_test, final_predictions)
mse = mean_squared_error(y_test, final_predictions)
r2 = r2_score(y_test, final_predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared: {r2}")

This approach provides a step-by-step guideline for stacking. In practice, the choice of base models and the meta-model can be varied based on the problem and data. Model stacking can be a powerful technique, but it also increases the complexity of the solution, so careful consideration is needed when deploying such models.

After executing the code, evaluate the performance metrics of the stacked model against the individual models to see if there's an improvement.